<a href="https://colab.research.google.com/github/erlonL/series-temporais/blob/main/ST_Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance snscrape pandas_market_calendars pandas_datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 10.6 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138891 sha256=9c21c6f97cfccd020853f95cb1f53bcf493ae17106c0a497f80c673ce4eec20a
  Stored in directory: /home/erlo/.cache/pip/wheels/a6/5e/0f/8319805c4115320e0d3e8fb5799b114a2e4c4a3d6c7e523b06
Successfully built peewee


In [9]:
import yfinance as yf
#import snscrape.modules.twitter as sntwitter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_market_calendars as mcal
import pandas_datareader.data as web

In [26]:
import snscrape

In [28]:
from transformers import pipeline

/home/erlo/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-20 09:41:23.434615: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 09:41:23.442938: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 09:41:23.530854: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 09:41:23.608662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory fo

In [30]:
new_model = pipeline(
    model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    return_all_scores=True
)

## ETAPA 1: Coleta e Tratamento de Dados

S&P 500, abreviação de Standard & Poor's 500, ou simplesmente S&P, trata-se de um índice composto por quinhentos ativos cotados nas bolsas de NYSE ou NASDAQ, qualificados devido ao seu tamanho de mercado, sua liquidez e sua representação de grupo industrial.

In [6]:
# Defina o ticker para o índice S&P 500
ticker = '^BVSP'

# Crie um objeto Ticker
index = yf.Ticker(ticker)

# Defina o intervalo de datas
start_date = '2022-07-01'
end_date = '2023-03-30'

# Obtenha os dados históricos
data = index.history(start=start_date, end=end_date)

# deixar apenas a data (y/m/d) na coluna date
data.index = data.index.date

# Exiba os primeiros registros para verificação
print(data.head())

               Open      High      Low     Close    Volume  Dividends  \
2022-07-01  98542.0   99340.0  97231.0   98954.0  11609800        0.0   
2022-07-04  98952.0   99353.0  98264.0   98609.0   6279300        0.0   
2022-07-05  98608.0   98608.0  96499.0   98295.0  13358800        0.0   
2022-07-06  98294.0   99141.0  97423.0   98719.0  13348200        0.0   
2022-07-07  98722.0  101420.0  98722.0  100730.0  12696300        0.0   

            Stock Splits  
2022-07-01           0.0  
2022-07-04           0.0  
2022-07-05           0.0  
2022-07-06           0.0  
2022-07-07           0.0  


No artigo, foi filtrado os dados para os dias úteis do mercado. Faremos a seguir esse processamento:

In [7]:
nyse = mcal.get_calendar('NYSE')

dias_uteis = nyse.valid_days(start_date=start_date, end_date=end_date)

# Pegar so a data
dias_uteis = [dia.date() for dia in dias_uteis]

In [8]:
sp500 = data.loc[data.index.intersection(dias_uteis)]

print(sp500.head())

sp500.to_csv("sp500.csv")

                Open      High      Low     Close    Volume  Dividends  \
2022-07-01   98542.0   99340.0  97231.0   98954.0  11609800        0.0   
2022-07-05   98608.0   98608.0  96499.0   98295.0  13358800        0.0   
2022-07-06   98294.0   99141.0  97423.0   98719.0  13348200        0.0   
2022-07-07   98722.0  101420.0  98722.0  100730.0  12696300        0.0   
2022-07-08  100732.0  101577.0  99958.0  100289.0   9730400        0.0   

            Stock Splits  
2022-07-01           0.0  
2022-07-05           0.0  
2022-07-06           0.0  
2022-07-07           0.0  
2022-07-08           0.0  


In [9]:
sp500.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,181.000000,181.000000,181.000000,181.000000,1.810000e+02,181.0,181.0
mean,108481.872928,109537.839779,107416.430939,108510.370166,1.378456e+07,0.0,0.0
std,5384.554239,5440.204289,5262.758184,5347.174473,3.542855e+06,0.0,0.0
min,96119.000000,96971.000000,95267.000000,96121.000000,0.000000e+00,0.0,0.0
25%,104375.000000,105497.000000,103409.000000,104385.000000,1.170710e+07,0.0,0.0
50%,109283.000000,110362.000000,108378.000000,109442.000000,1.338910e+07,0.0,0.0
75%,112307.000000,113306.000000,111380.000000,112323.000000,1.548190e+07,0.0,0.0
max,119922.000000,120752.000000,117144.000000,119929.000000,2.602930e+07,0.0,0.0


In [10]:
query = "S&P500"
min_likes = 0
xlsx_name = "tweets_sp500.xlsx"

In [18]:
# search query for reddit

posts = []

for i, post in enumerate(snscrape.modules.reddit.RedditSearchScraper(query).get_items()):
    print(i, post.title, post.url, post.likes, post.num_comments, post.created)
    posts.append([post.title, post.url, post.likes, post.num_comments, post.created])

df = pd.DataFrame(posts, columns=["title", "url", "likes", "num_comments", "created"])
df

Error retrieving https://api.pushshift.io/reddit/search/submission?q=S%26P500&limit=1000: non-200 status code
4 requests to https://api.pushshift.io/reddit/search/submission?q=S%26P500&limit=1000 failed, giving up.
Errors: non-200 status code, non-200 status code, non-200 status code, non-200 status code


ScraperException: 4 requests to https://api.pushshift.io/reddit/search/submission?q=S%26P500&limit=1000 failed, giving up.

In [11]:
tweets = []

search_query = f'{query} min_retweets:0 min_faves:{min_likes} since:{start_date} until:{end_date}'

for tweet in sntwitter.TwitterSearchScraper(search_query).get_items():
    # Extract relevant information from the tweet
    tweet_data = {
        "tweet": tweet.content,
        "Date": tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
    }

    tweets.append(tweet_data)

tweets

Error retrieving https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2023-03-30&src=spelling_expansion_revert_click: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'twitter.com\', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2023-03-30&src=spelling_expansion_revert_click (Caused by NameResolutionError("<snscrape.base._HTTPSConnection object at 0x7aed9a6e3280>: Failed to resolve \'twitter.com\' ([Errno -2] Name or service not known)"))'))
4 requests to https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2023-03-30&src=spelling_expansion_revert_click failed, giving up.
Errors: ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'twitter.com\', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+un

ScraperException: 4 requests to https://twitter.com/search?f=live&lang=en&q=S%26P500+min_retweets%3A0+min_faves%3A0+since%3A2022-07-01+until%3A2023-03-30&src=spelling_expansion_revert_click failed, giving up.

# Etapa 2: Análise e Agregação de Sentimento através do DistilRoBERTa